# Introduction (Himmelblau's function)



Let's use ``bloptools`` to minimize Himmelblau's function, which has four global minima:

In [ ]:
from bloptools.utils import prepare_re_env

%run -i $prepare_re_env.__file__ --db-type=temp

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from bloptools import test_functions

x1 = x2 = np.linspace(-6, 6, 1024)
X1, X2 = np.meshgrid(x1, x2)

F = test_functions.himmelblau(X1, X2)

plt.pcolormesh(x1, x2, F, norm=mpl.colors.LogNorm(vmin=1e-1, vmax=1e3), cmap="magma_r")
plt.colorbar()
plt.xlabel("x1")
plt.ylabel("x2")

There are several things that our agent will need. The first ingredient is some degrees of freedom (these are always `ophyd` devices) which the agent will move around to different inputs within each DOF's bounds (the second ingredient). We define these here:

In [ ]:
from bloptools.bayesian import DOF, BrownianMotion

dofs = [
    DOF(name="x1", limits=(-6, 6)),
    DOF(name="x2", limits=(-6, 6)),
    DOF(BrownianMotion(name="brownian1"), read_only=True),
    DOF(BrownianMotion(name="brownian2"), read_only=True),
]

We also need to give the agent something to do. We want our agent to look in the feedback for a variable called "himmelblau", and try to minimize it.

In [ ]:
from bloptools.bayesian import Objective

objectives = [Objective(key="himmelblau", minimize=True)]

In our digestion function, we define our objective as a deterministic function of the inputs:

In [ ]:
def digestion(db, uid):
    products = db[uid].table()

    for index, entry in products.iterrows():
        products.loc[index, "himmelblau"] = test_functions.himmelblau(entry.x1, entry.x2)

    return products

We then combine these ingredients into an agent, giving it an instance of ``databroker`` so that it can see the output of the plans it runs.

In [ ]:
from bloptools.bayesian import Agent


agent = Agent(
    dofs=dofs,
    objectives=objectives,
    digestion=digestion,
    db=db,
)

In [ ]:
agent.test_inputs_grid().shape

Without any data, we can't make any inferences about what the function looks like, and so we can't use any non-trivial acquisition functions. Let's start by quasi-randomly sampling the parameter space, and plotting our model of the function:

In [ ]:
RE(agent.learn("quasi-random", n=32))
agent.plot_objectives()

To decide which points to sample, the agent needs an acquisition function. The available acquisition function are here:

In [ ]:
agent.acq_func_info

Now we can start to learn intelligently. Using the shorthand acquisition functions shown above, we can see the output of a few different ones:

In [ ]:
agent.plot_acquisition(acq_funcs=["qei", "pi", "qucb"])

In [ ]:
agent.dofs.limits

In [ ]:
self = agent
import torch

acq_func_lower_bounds = [dof.lower_limit if not dof.read_only else dof.readback for dof in self.dofs]
acq_func_upper_bounds = [dof.upper_limit if not dof.read_only else dof.readback for dof in self.dofs]

torch.tensor(np.vstack([acq_func_lower_bounds, acq_func_upper_bounds]))

To decide where to go, the agent will find the inputs that maximize a given acquisition function:

In [ ]:
agent.acquisition_function_bounds

In [ ]:
agent.ask("qei", n=1)

We can also ask the agent for multiple points to sample and it will jointly maximize the acquisition function over all sets of inputs, and find the most efficient route between them:

In [ ]:
X, _ = agent.ask("qei", n=8, route=True)
agent.plot_acquisition(acq_funcs=["qei"])
plt.scatter(*X.T, marker="d", facecolor="w", edgecolor="k")
plt.plot(
    *X.T,
    color="r",
)

All of this is automated inside the ``learn`` method, which will find a point (or points) to sample, sample them, and retrain the model and its hyperparameters with the new data. To do 4 learning iterations of 8 points each, we can run

In [ ]:
RE(agent.learn("qei", n=8, iterations=4))

Our agent has found all the global minima of Himmelblau's function using Bayesian optimization, and we can ask it for the best point: 

In [ ]:
agent.plot_objectives()
# print(agent.best_inputs)